##Climate Change Dashboard

### Dataset: Using the Climate API

In [0]:
import pandas as pd
url = "https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/global/time-series/globe/tavg/land_ocean/ytd/0/1850-2025/data.csv"

Global_temp=pd.read_csv(url,skiprows=4,names=['YearMonth','Value'])

Global_temp["Year"]=Global_temp["YearMonth"]//100
Global_temp["Month"]=Global_temp["YearMonth"]%100

Global_temp=Global_temp[['Year','Month','Value']]

df = spark.createDataFrame(Global_temp)

#Setting the catalog to be used 
spark.sql("USE CATALOG workspace")
spark.sql("USE SCHEMA climate_data")

df.write.mode("ignore").saveAsTable("global_temp_per_year")
display(spark.table("global_temp_per_year"))

Bar chart

In [0]:
<!DOCTYPE html>
<html lang="en">

<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Record Number of Days Above 1.5°C in 2023</title>
  <script src="https://d3js.org/d3.v7.min.js"></script>
  <style>
    body {
      font-family: Arial, sans-serif;
      background-color: black;
      /* Background color of the page */
      color: white;
      /* Text color for the body */
    }

    svg {
      background-color: black;
      /* Background color for the chart */
    }

    .title {
      font-weight: bold;
      font-size: 18px;
      text-anchor: middle;
      fill: white;
      /* Title color */
    }

    .annotation {
      font-size: 12px;
      fill: white;
      /* Annotation text color */
      font-weight: bold;
      text-anchor: middle;
    }

    .legend {
      font-size: 12px;
      fill: white;
      /* Legend text color */
    }

    .credit {
      font-size: 10px;
      fill: gray;
    }

    /* Style for axis lines and labels */
    .axis path,
    .axis line {
      stroke: white;
      /* Axis lines color */
    }

    .axis text {
      fill: white;
      /* Axis labels color */
    }

    .dashed-line {
      stroke: white;
      stroke-dasharray: 5,5;
      stroke-width: 2;
    }
  </style>
</head>

<body>
  <script>
    // Chart dimensions and margins
    const margin = { top: 50, right: 30, bottom: 50, left: 60 };
    const width = 900 - margin.left - margin.right;
    const height = 500 - margin.top - margin.bottom;

    // Create SVG container
    const svg = d3.select("body")
      .append("svg")
      .attr("width", width + margin.left + margin.right)
      .attr("height", height + margin.top + margin.bottom)
      .append("g")
      .attr("transform", `translate(${margin.left},${margin.top})`);

    // Load CSV data
    d3.csv("daily_global_temperature_increase_above_preindustrial_2023 (3).csv").then(data => {
      // Parse data and convert to numeric
      data.forEach(d => {
        d.year = +d.year;
        d["Over 1.5"] = +d["Over 1.5"];
        d["1.25 to 1.5"] = +d["1.25 to 1.5"];
        d["1 to 1.25"] = +d["1 to 1.25"];
        d.total = d["Over 1.5"] + d["1.25 to 1.5"] + d["1 to 1.25"];
      });

      // Filter data for years starting from 1990
      const filteredData = data.filter(d => d.year >= 1990);

      // Define scales
      const x = d3.scaleBand()
        .domain(filteredData.map(d => d.year))
        .range([0, width])
        .padding(0.2);

      const y = d3.scaleLinear()
        .domain([0, d3.max(filteredData, d => d.total)])
        .nice()
        .range([height, 0]);

      const colors = {
        "1 to 1.25": "#FFC4A3",
        "1.25 to 1.5": "#FF7F50",
        "Over 1.5": "#B22222"
      };

      // Add X axis
      svg.append("g")
        .attr("transform", `translate(0,${height})`)
        .call(d3.axisBottom(x).tickFormat(d3.format("d")))
        .selectAll("text")
        .attr("transform", "rotate(45)")
        .style("text-anchor", "start");

      // Add Y axis
      svg.append("g")
        .call(d3.axisLeft(y));

      // Stack data
      const stack = d3.stack()
        .keys(["1 to 1.25", "1.25 to 1.5", "Over 1.5"])
        .value((d, key) => d[key]);

      const series = stack(filteredData);

      // Add bars
      svg.selectAll(".layer")
        .data(series)
        .enter().append("g")
        .attr("fill", d => colors[d.key])
        .selectAll("rect")
        .data(d => d)
        .enter().append("rect")
        .attr("x", d => x(d.data.year))
        .attr("y", d => y(d[1]))
        .attr("height", d => y(d[0]) - y(d[1]))
        .attr("width", x.bandwidth());

      // Add title
      svg.append("text")
        .attr("x", width / 2)
        .attr("y", -20)
        .attr("class", "title")
        .text("RECORD NUMBER OF DAYS ABOVE 1.5°C OVER THE YEARS");

      // Add a dashed line at y = 365
      const dashedLineY = y(365);

      // Add the dashed line
      svg.append("line")
        .attr("x1", 0)
        .attr("y1", dashedLineY)
        .attr("x2", width)
        .attr("y2", dashedLineY)
        .attr("stroke", "white")
        .attr("stroke-dasharray", "5,5") // Dashed line pattern
        .attr("stroke-width", 2)
        .attr("class", "dashed-line");

      // Add annotation for the dashed line
      svg.append("text")
        .attr("x", width - 10)
        .attr("y", dashedLineY - 5)
        .attr("class", "annotation")
        .attr("text-anchor", "end")
        .text("365 Days");

      // Add legend
      const legend = svg.append("g")
        .attr("class", "legend")
        .attr("transform", `translate(${width / 2 - 100}, ${dashedLineY - 30})`); // Position above the dashed line

      const keys = ["1 to 1.25", "1.25 to 1.5", "Over 1.5"];
      const legendSpacing = 20;

      // Add legend items horizontally with "°C" added to the legend text
      keys.forEach((key, i) => {
        legend.append("rect")
          .attr("x", i * 100) // Horizontally spaced
          .attr("y", 0) // Same vertical alignment
          .attr("width", 15)
          .attr("height", 15)
          .attr("fill", colors[key]);

        legend.append("text")
          .attr("x", i * 100 + 20) // Spaced to the right of the rectangle
          .attr("y", 12) // Vertically aligned with the rectangle
          .text(key + " °C")  // Add "°C" to the legend text
          .attr("class", "legend");
      });

    });
  </script>

</body>

</html>

